In [1]:
import ipywidgets as ipw
from home.app_store import AiidaLabAppStore

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
display(ipw.HTML("""/<a href="start.ipynb"> home </a>/ appstore"""))


HTML(value='/<a href="start.ipynb"> home </a>/ appstore')

# <center>App Store</center>

In [4]:
control = AiidaLabAppStore()
display(control)
display(ipw.HBox([control.items_per_page, control.page_selector]))
display(control.render())
display(ipw.HBox([control.items_per_page, control.page_selector]))

AiidaLabAppStore(children=(Checkbox(value=False, description='Show only installed'), VBox(children=(SelectMult…

Output()